In [ ]:
import os
from glob import glob
import matplotlib.pyplot as plt
import cv2
import numpy as np
import h5py
from ipywidgets import interact

In [ ]:
def gen_list_of_image_names(abs_path, extension):
    list_of_image_names = glob(os.path.join(abs_path, '*'+extension))
    return list_of_image_names

In [ ]:
def gen_dateset(list_of_image_names, square=False, newsize=None, color=False):
    dataset = []
    for img_name in list_of_image_names:
        if color:
            img = cv2.imread(img_name,1) #array 720x1280 rgb
        else:
            img = cv2.imread(img_name,0) #array 720x1280 grayscale
        if (img is None):
            print(img_name,'non eksiste')
        else:
            if square:
                img = img[:,280:1000] #subentendendo que a imagem ori eh 720x1280
            if newsize:
                img = cv2.resize(img, newsize) 
            dataset.append(img)
    #print(type(img))
#     print(len(dataset))
#     print(type(dataset))
    dataset = np.stack(dataset)
    return dataset

In [ ]:
def permutate(ds, labels=None):
    p = np.random.permutation(ds.shape[0])
    ds = ds[p]
    if (labels is not None):
        labels = labels[p]
        return ds, labels
    return ds

In [ ]:
def gen_targets(n):
    return tuple([np.identity(n)[i,:] for i in range(n)])

In [ ]:
path = '/Users/henriquegoncalves/Documents/UFRJ/PF2/Images/imagens_v1/'
cl_names = ['dano', 'variacao', 'duto']
cl_paths = {}

for i in range(len(cl_names)):
    cl_paths[cl_names[i]] = gen_list_of_image_names(os.path.join(path,cl_names[i]),'.jpg')

#print(cl_paths['dano'][324])

Separando quais videos são treino, teste e validação

In [ ]:
test_video_name = 'HD_C_pl0138_VOL006'
valid_video_name = 'HD_C_pl0138_VOL007'
te_paths = {}
tr_paths = {}
v_paths = {}

for i in range(len(cl_names)):
    te_paths[cl_names[i]] = [path for path in cl_paths[cl_names[i]] if test_video_name in path]
    v_paths[cl_names[i]] = [path for path in cl_paths[cl_names[i]] if valid_video_name in path]

    tr_paths[cl_names[i]] = [path for path in cl_paths[cl_names[i]] if test_video_name not in path]
    tr_paths[cl_names[i]] = [path for path in tr_paths[cl_names[i]] if valid_video_name not in path]

Gerar os arquivos de dataset

In [ ]:
%%time
te_data = {}
tr_data = {}
v_data = {}

for i in range(len(cl_names)):
    te_data[cl_names[i]] = gen_dateset(te_paths[cl_names[i]],square=True, newsize=(224,224), color=True)
    tr_data[cl_names[i]] = gen_dateset(tr_paths[cl_names[i]],square=True, newsize=(224,224), color=True)
    v_data[cl_names[i]] = gen_dateset(v_paths[cl_names[i]],square=True, newsize=(224,224), color=True)


In [ ]:
print(tr_data['variacao'].shape[0],tr_data['duto'].shape[0],tr_data['dano'].shape[0])
print(te_data['variacao'].shape[0],te_data['duto'].shape[0],te_data['dano'].shape[0])
print(v_data['variacao'].shape[0],v_data['duto'].shape[0],v_data['dano'].shape[0])

Balancear as classes

In [ ]:
tr_list = []
te_list = []
v_list = []

for cl_name in cl_names:
    tr_list.append(tr_data[cl_name].shape[0])
    te_list.append(te_data[cl_name].shape[0])
    v_list.append(v_data[cl_name].shape[0])

tr_m = min(tr_list)
te_m = min(te_list)
v_m = min(v_list)
    
for cl_name in cl_names:
    tr_data[cl_name] = permutate(tr_data[cl_name])
    te_data[cl_name] = permutate(te_data[cl_name])
    v_data[cl_name] = permutate(v_data[cl_name])
    
for cl_name in cl_names:
    tr_data[cl_name] = tr_data[cl_name][:tr_m]
    v_data[cl_name] = v_data[cl_name][:v_m]
    te_data[cl_name] = te_data[cl_name][:te_m]

    
    
# tr_m = min(tr_cl_data.shape[0], tr_ncl_data.shape[0])
# te_m = min(te_cl_data.shape[0], te_ncl_data.shape[0])
# v_m = min(v_cl_data.shape[0], v_ncl_data.shape[0])

# tr_cl_data = permutate(tr_cl_data)
# te_cl_data = permutate(te_cl_data)
# v_cl_data = permutate(v_cl_data)
# tr_ncl_data = permutate(tr_ncl_data)
# te_ncl_data = permutate(te_ncl_data)
# v_ncl_data = permutate(v_ncl_data)

# tr_cl_data = tr_cl_data[:tr_m]
# te_cl_data = te_cl_data[:te_m]
# v_cl_data = v_cl_data[:v_m]
# tr_ncl_data = tr_ncl_data[:tr_m]
# te_ncl_data = te_ncl_data[:te_m]
# v_ncl_data = v_ncl_data[:v_m]

# print(tr_cl_data.shape, tr_ncl_data.shape, te_cl_data.shape, te_ncl_data.shape, v_cl_data.shape, v_ncl_data.shape)

In [ ]:
print([te_data[i].shape[0] for i in cl_names])

In [ ]:
n = len(cl_names)
targets = gen_targets(n)

X_tr = np.concatenate(tuple([tr_data[cl_name] for cl_name in cl_names]))
Y_tr = np.concatenate(tuple([np.tile(targets[i],(tr_m,1)) for i in range(n)]))

X_v = np.concatenate(tuple([v_data[cl_name] for cl_name in cl_names]))
Y_v = np.concatenate(tuple([np.tile(targets[i],(v_m,1)) for i in range(n)]))

X_te = np.concatenate(tuple([te_data[cl_name] for cl_name in cl_names]))
Y_te = np.concatenate(tuple([np.tile(targets[i],(te_m,1)) for i in range(n)]))


# X_tr = np.concatenate((tr_cl_data, tr_ncl_data))
# Y_tr = np.concatenate((np.ones((tr_m,1)),np.zeros((tr_m,1))))

# X_te = np.concatenate((te_cl_data, te_ncl_data))
# Y_te = np.concatenate((np.ones((te_m,1)),np.zeros((te_m,1))))

# X_v = np.concatenate((v_cl_data, v_ncl_data))
# Y_v = np.concatenate((np.ones((v_m,1)),np.zeros((v_m,1))))

X_tr, Y_tr = permutate(X_tr, Y_tr)
X_v, Y_v = permutate(X_v, Y_v)
X_te, Y_te = permutate(X_te, Y_te)

classes = [cl_name.encode() for cl_name in cl_names]

In [ ]:
xgroup = X_tr
ygroup = Y_tr

def interactive_print(index):
    plt.figure(figsize=(6,6))
    plt.imshow(xgroup[index,:,:,:])
    print('This image is of class:',classes[np.argmax(ygroup[index,:])].decode())
print(len(ygroup))
interact(interactive_print, index = (0,len(xgroup)))

In [ ]:
path = '../../datasets/'
filename = 'softmaxVaDaDu_224color.h5'
f = os.path.join(path, filename)

h5f = h5py.File(f, 'w')
h5f.create_dataset('X_tr', data=X_tr)
h5f.create_dataset('Y_tr', data=Y_tr)
h5f.create_dataset('X_v', data=X_v)
h5f.create_dataset('Y_v', data=Y_v)
h5f.create_dataset('X_te', data=X_te)
h5f.create_dataset('Y_te', data=Y_te)
h5f.create_dataset('classes', data=classes)
h5f.close()

Normalização

In [ ]:
import pandas as pd

In [ ]:
#names = gen_list_of_image_names('/Users/henriquegoncalves/Documents/UFRJ/PF2/Images/imagens/dano','.jpg')
ds = os.listdir('/Users/henriquegoncalves/Documents/UFRJ/PF2/Images/imagens/flange/')
df = pd.DataFrame({'classe': ds})
# df = pd.DataFrame({'dano': dano})
df['video'] = df.apply(lambda row: row.classe[:18],axis=1)
print(len(df))
df.groupby(['video'],sort=False).count()

# df['dano']
# video_path = '/Users/henriquegoncalves/Documents/UFRJ/PF2/Videos/'
# videos = [i for i in os.listdir(video_path) if i.endswith(".mp4")]
# for video in videos:
#     video = video.split('.')[0]
# videos = [video.split('.')[0] for video in videos]

# 06 teste, 07 valid

In [ ]:
print(len(df)-1411-1317)

In [ ]:
543+537+597+600+648+666+645+585+519+27#+747+609

In [ ]:
targets = gen_tergets(3)
np.tile(targets[0],(3,1))

In [ ]:
m=3
np.concatenate((np.ones((m,1)),np.zeros((m,1))))

In [ ]:
m=3
n = len(cl_names)
targets = gen_tergets(n)
np.concatenate(tuple([np.tile(targets[i],(m,1)) for i in range(n)]))
# np.concatenate((np.tile(targets[0],(m,1)),np.tile(targets[1],(m,1)),np.tile(targets[2],(m,1))))

In [ ]:
index = 0
print(np.argmax(ygroup[index,:]))
print(classes[int(ygroup[index,0])].decode())
print(classes)

In [ ]:
np.argmax(gen_targets(3)[1])